<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%mkdir spark

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -P /spark https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-20 13:24:14--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 140.211.166.134, 64.50.233.100, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|140.211.166.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  18.4MB/s    in 2m 37s  

2021-07-20 13:26:51 (1.37 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz


In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"


In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
myRange = spark.range(1000).toDF("number")

In [9]:
myRange.show(10)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
+------+
only showing top 10 rows



In [10]:
#find all even numbers in myRange
dvBy2 = myRange.where("number % 2 = 0")
dvBy2.show(10)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
+------+
only showing top 10 rows



In [11]:
!wget -P /sample_data/ https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv

--2021-07-20 13:27:12--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7080 (6.9K) [text/plain]
Saving to: ‘/sample_data/2015-summary.csv’

2015-summary.csv    100%[===================>]   6.91K  --.-KB/s    in 0s      

2021-07-20 13:27:12 (77.7 MB/s) - ‘/sample_data/2015-summary.csv’ saved [7080/7080]



In [12]:
%cd /sample_data/
%mv 2015-summary.csv.1 2015-summary.csv
%ls

/sample_data
mv: cannot stat '2015-summary.csv.1': No such file or directory
2015-summary.csv


In [13]:
#Reading from the file, inferring the schema and also including the header
from pyspark.sql.functions import to_timestamp,col,lit
#rc = spark.read.csv('/sample_data/2015-summary.csv', header=True).show(10)
flightData2015 = spark \
. read \
. option ( "inferSchema" , "true" ) \
. option ( "header" , "true" ) \
. csv ( "/sample_data/2015-summary.csv" )




In [14]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#32 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#32 ASC NULLS FIRST, 200), true, [id=#54]
   +- FileScan csv [DEST_COUNTRY_NAME#30,ORIGIN_COUNTRY_NAME#31,count#32] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/sample_data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [16]:
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [17]:
flightData2015.createOrReplaceTempView("flightData2015")

In [18]:
flightData2015.groupBy("origin_country_name")\
.count()\
.show(10)\
#flightData2015.show(10)
spark.sql("select origin_country_name, count(*) from flightData2015 group by  origin_country_name").show(10)

+--------------------+-----+
| origin_country_name|count|
+--------------------+-----+
|             Ireland|    1|
|           Singapore|    1|
|    Marshall Islands|    1|
|              Angola|    1|
|            Portugal|    1|
|            Suriname|    1|
|          Cape Verde|    1|
|            Malaysia|    1|
|              Panama|    1|
|Turks and Caicos ...|    1|
+--------------------+-----+
only showing top 10 rows

+--------------------+--------+
| origin_country_name|count(1)|
+--------------------+--------+
|             Ireland|       1|
|           Singapore|       1|
|    Marshall Islands|       1|
|              Angola|       1|
|            Portugal|       1|
|            Suriname|       1|
|          Cape Verde|       1|
|            Malaysia|       1|
|              Panama|       1|
|Turks and Caicos ...|       1|
+--------------------+--------+
only showing top 10 rows



In [28]:
flightData2015.show(1)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+
only showing top 1 row



In [ ]:

from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

In [42]:
sumOfDestCountry = spark.sql("select DEST_COUNTRY_NAME, sum(count) as dest_total from  flightData2015 group by DEST_COUNTRY_NAME order by sum(count) desc limit 5 ")
sumOfDestCountry.show()


+-----------------+----------+
|DEST_COUNTRY_NAME|dest_total|
+-----------------+----------+
|    United States|    411352|
|           Canada|      8399|
|           Mexico|      7140|
|   United Kingdom|      2025|
|            Japan|      1548|
+-----------------+----------+



In [70]:
#With import of desc function
from pyspark.sql.functions import desc 
getSum = flightData2015.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "dest_total")\
.sort(desc("dest_total"))

getSum
  

DataFrame[DEST_COUNTRY_NAME: string, dest_total: bigint]

In [71]:
#Without import of desc function
getSum = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")


In [64]:
getSum.sort(getSum.destination_total.desc()).take(5)

[Row(DEST_COUNTRY_NAME='United States', destination_total=411352),
 Row(DEST_COUNTRY_NAME='Canada', destination_total=8399),
 Row(DEST_COUNTRY_NAME='Mexico', destination_total=7140),
 Row(DEST_COUNTRY_NAME='United Kingdom', destination_total=2025),
 Row(DEST_COUNTRY_NAME='Japan', destination_total=1548)]